In [4]:
import pandas as pd
import numpy as np
import csv as csv
import re

from glob import glob

In [5]:
# list paths to software and locations we will need later
idba_ud = '/shared/software/bin/idba_ud'
mash = '/shared/software/bin/mash'

# Define projects

In [6]:
# AWTP2:
project = 'awtp2'
df = pd.read_csv('/Users/rosekantor/data/awtp2/metagenomics/tables/sample_info.csv')
df = df.rename(columns={'ggkbase_project_name':'sample_id'})
df['combined_reads'] = f'/groups/banfield/sequences/2022/' + df.sample_id + '/raw.d/' + df.sample_id + '_trim_clean.PE.fa'

# assign paths
assem_dir = f'/groups/banfield/projects/industrial/nelson_lab/awtp2/assemblies'
reads_dir = f'/groups/banfield/projects/industrial/nelson_lab/awtp2/reads/trimmed'
mash_dir = f'/groups/banfield/projects/industrial/nelson_lab/awtp2/mash_analysis'
outdir = f'/Users/rosekantor/data/awtp2/metagenomics/workflow/'

# Raw read processing

In [4]:
df['name'] = df.sample_id
df['contigs_db_path'] = f'{assem_dir}/' + df.sample_id + '.idba_ud/' + df.sample_id + '_contigs.db'
df['profile_db_path'] = f'{assem_dir}/' + df.sample_id + '.idba_ud/anvio_data/' + df.sample_id + '_profile/PROFILE.db'

In [5]:
metagenome_table = df[~df['name'].str.contains('CTRL')][['name', 'contigs_db_path', 'profile_db_path']]
metagenome_table.to_csv(f'{outdir}/metagenome_table.tsv', sep='\t', index=False)

1. Run `bbmap` and `sickle` on raw reads.

2. Make symlinks to reads in our analysis folders using `ln -s` so that we can have easy access to them in the future.

3. Review the output of fastqc to check the quality of the reads.  If reads appear to be of low or questionable quality, run fastqc on the trimmed reads to check that quality of the remaining reads is higher after trimming and any concerning artifacts have been removed.

4. Count the reads before and after trimming. From within /trimmed and /raw, open a tmux session and run this command for counting reads:
    
`seqkit stat *.fastq.gz > seqkit_output.tsv`

5. Combine the sample project names table with the output table from seqkit to track the data for each sample.  How much total sequencing per sample, how many forward/reverse reads before/after trimming?

# Assembly commands

Using the sample table, generate commands for running assemblies on the cluster (via SLURM). Commands may specify whether to use the memory or high-memory nodes.

In [170]:
# we will create an empty list, iterate thru the df and generate commands, save them in the list, then turn the list into a column in our df
idba_cmd = []
for row in df.itertuples():
    # some samples may require more memory for assembly
    if 'INF' in row.sample_id or 'BAC' in row.sample_id:
        cmd = f'sbatch --partition memory --wrap "' \
              f'{idba_ud} ' \
              f'--pre_correction -r {row.combined_reads} ' \
              f'-o {assem_dir}/{row.sample_id}.idba_ud"'
    else:
        cmd = f'sbatch --wrap "' \
          f'{idba_ud} ' \
          f'--pre_correction -r {row.combined_reads} ' \
          f'-o {assem_dir}/{row.sample_id}.idba_ud"'

    idba_cmd.append(cmd)
df['idba_cmd'] = idba_cmd

# write the commands to a CSV
df['idba_cmd'].to_csv(f'{outdir}/1.assembly.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

note: alternatively you could just open a file and write the commands directly to the file as you create them (in the for loop). You could also open a jupyter notebook on biotite and run the commands directly from there (not recommended because it's harder to track what is running).

Now push this shell script up to biotite and run it: `sh assembly.sh`

# Mash commands

Compare samples to each other using minhash distances between reads files.  First, need to combine forward and reverse reads into a single stream and then create a mash sketch for each sample.  Then all samples can be pairwise compared to each other to produce a distance matrix.

In [171]:
# mash all v. all reads
mash_cmd = []
for row in df.itertuples():
    cmd = f'cat {reads_dir}/{s}_trim_clean.PE.1.fastq.gz {reads_dir}/{s}_trim_clean.PE.2.fastq.gz | {mash} sketch -m 2 -r - -I {row.sample_id} -s 10000 -o {mash_dir}/{row.sample_id}'
    mash_cmd.append(cmd)
    
df['mash_cmd'] = mash_cmd

In [172]:
# write the commands to a CSV
df['mash_cmd'].to_csv(f'{outdir}/2.mash.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

## mash dist commands
`mash paste` combines multiple sketches into a single sketch.  The first arg is output name, followed by a list of all the sketch files you want to combine

Command: `mash paste awtp2.msh *msh`

`mash dist` can sketch on the fly or take a sketch as input.  Because we are doing all-vs-all we use the same msh file as the query and reference.

Command: `mash dist awtp2.msh awtp2.msh`

# Post-assembly commands

1. Rename the scaffolds so that they have the project name in the fasta headers.
2. Check assembly quality using Itai Sharon's contig_stats.pl 
3. Filter scaffolds > 1000 bp into a new min1000.fa file for further analysis.
4. Remove extra files created by the assembler
5. Make bowtie2 indexes in their own folder

In [177]:
postassem_cmd = [] # create empty list
for row in df.itertuples():
    s = row.sample_id
    
    scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_scaffold.fa'

    # include "sample_id" in headers and rename file to sample_id_scaffold.fa
    rehead = f"sed 's/scaffold/{s}/g' {assem_dir}/{s}.idba_ud/scaffold.fa > {scaffolds}"

    # get contig stats
    cstats = f'contig_stats.pl -i {scaffolds}'

    # delete extra files from assembly

    clean = f'rm {assem_dir}/{s}.idba_ud/kmer '\
            f'{assem_dir}/{s}.idba_ud/contig-* '\
            f'{assem_dir}/{s}.idba_ud/align-* '\
            f'{assem_dir}/{s}.idba_ud/graph-* '\
            f'{assem_dir}/{s}.idba_ud/local-*'

    # make directory to store bowtie2 indices in
    mdbt2 = f'mkdir {assem_dir}/{s}.idba_ud/bt2/'

    # index in prep for bowtie2 mapping to get true coverage for ggkbase
    ind = f'bowtie2-build {scaffolds} {assem_dir}/{s}.idba_ud/bt2/{s}_scaffold.fa'
    
    cmd = [rehead, cstats, clean, mdbt2, ind]
    all_cmd = '; '.join(cmd) # separate all commands by semicolon (so they will be executed in order for each sample)
    postassem_cmd.append(all_cmd) # append command to list
    
df['postassem_cmd'] = postassem_cmd # add as a column to the dataframe

In [178]:
# write the commands to a CSV
df['postassem_cmd'].to_csv(f'{outdir}/3.postassem.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

# ggkbase import: 
## Mapping and gene-calling

In [63]:
shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'

self_map_cmd = []
readcounts_cmd = []
genecalls_cmd = []

for row in df.itertuples():
    
    s = row.sample_id
    bt2base = f'{assem_dir}/{s}.idba_ud/bt2/{s}_scaffold.fa'
    r1 = f'{reads_dir}/{s}_trim_clean.PE.1.fastq.gz'
    r2 = f'{reads_dir}/{s}_trim_clean.PE.2.fastq.gz'
    scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_scaffold.fa'
    scaffolds_min1000 = f'{assem_dir}/{s}.idba_ud/{s}_scaffold_min1000.fa'

    # perform self-mapping and add readcounts to scaffold headers
    map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} 2> {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.log | {shrinksam} -v > {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.sam"'
    self_map_cmd.append(map_cmd)
    
    add_readcount = f'/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.sam {assem_dir}/{s}.idba_ud/{s}_scaffold.fa 150 > {assem_dir}/{s}.idba_ud/{s}_scaffold.fa.counted'
    move_file = f'mv {assem_dir}/{s}.idba_ud/{s}_scaffold.fa.counted {assem_dir}/{s}.idba_ud/{s}_scaffold.fa'
    
    # filter for only contigs ≥1000 bp
    min1000 = f'pullseq -i {scaffolds} --min 1000 > {scaffolds_min1000}'
    
    readcounts_cmd.append(f'{add_readcount}; {move_file}; {min1000}') # append command to list
    
    # gene-calling
    call_orfs = f'prodigal -i {scaffolds_min1000} -o {scaffolds_min1000}.genes -a {scaffolds_min1000}.genes.faa -d {scaffolds_min1000}.genes.fna -m -p meta'
    call_16S = f'/groups/banfield/software/pipeline/v1.1/scripts/16s.sh {scaffolds_min1000} > {scaffolds_min1000}.16s'
    call_trnas = f'/groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i {scaffolds_min1000} > /dev/null 2>&1'
    genecalls_cmd.append(f'{call_orfs}; {call_16S}; {call_trnas}')

df['self_map_cmd'] = self_map_cmd
df['readcounts_cmd'] = readcounts_cmd
df['genecalls_cmd'] = genecalls_cmd

In [64]:
# write the commands to a CSV
df['self_map_cmd'].to_csv(f'{outdir}/4.ggkbase_self_map.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['readcounts_cmd'].to_csv(f'{outdir}/5.ggkbase_readcounts.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['genecalls_cmd'].to_csv(f'{outdir}/6.ggkbase_genecalls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

## Annotation by best reciprocal blast

In [61]:
usearch = '/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb'
annolookup = '/shared/software/bin/annolookup.py'

usearch_cmds = []
anno_proccess_cmds = []

for row in df.itertuples():
    s = row.sample_id
    faa = f'{assem_dir}/{s}.idba_ud/{s}_scaffold_min1000.fa.genes.faa'
    
    # make usearch commands (using cluster usearch wrapper)
    search_kegg = f'sbatch --wrap "{usearch} -i {faa} -k -d kegg --nocluster"'
    search_uni = f'sbatch --wrap "{usearch} -i {faa} -k -d uni --nocluster"'
    search_uniprot = f'sbatch --wrap "{usearch} -i {faa} -k -d uniprot --nocluster"'
    
    usearch_cmds.append(f'{search_kegg}; {search_uni}; {search_uniprot}')

    # gzip files
    gzip_b6 = f'gzip {faa}*.b6'
    
    # annolookup
    anno_kegg = f'{annolookup} {faa}-vs-kegg.b6.gz kegg > {faa}-vs-kegg.b6+'
    anno_uni = f'{annolookup} {faa}-vs-uni.b6.gz uniref > {faa}-vs-uni.b6+'
    anno_uniprot = f'{annolookup} {faa}-vs-uniprot.b6.gz uniprot > {faa}-vs-uniprot.b6+'
    
    anno_proccess_cmds.append(f'{gzip_b6}; {anno_kegg}; {anno_uni}; {anno_uniprot}')

df['usearch_cmds'] = usearch_cmds
df['anno_proccess_cmds'] = anno_proccess_cmds    

In [62]:
# write the commands to a CSV
df['usearch_cmds'].to_csv(f'{outdir}/7.ggkbase_usearch.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['anno_proccess_cmds'].to_csv(f'{outdir}/8.ggkbase_anno_process.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

The files have now been generated for ggkbase import.

# Anvi'o process contigs

See https://merenlab.org/2016/06/18/importing-taxonomy/ and https://merenlab.org/2016/06/22/anvio-tutorial-v2/

In [60]:
awk = "awk '{print $1}'" # using this to remove additional fields from fasta headers for contigs, ignore if you don't have this issue
anvio = 'conda activate anvio-7.1' # note that this didn't actually work, needed to use conda run but we used system anvio v7

kaiju_path = '/groups/banfield/projects/industrial/nelson_lab/kaiju/bin'
kaiju_nodes = '/shared/db/kaiju/nr_euk/r2021-02-24/nodes.dmp'
kaiju_names = '/shared/db/kaiju/nr_euk/r2021-02-24/names.dmp'

kaiju_nr = '/shared/db/kaiju/nr_euk/r2021-02-24/kaiju_db_nr_euk.fmi'
kaiju = f'{kaiju_path}/kaiju -t {kaiju_nodes} -f {kaiju_nr}'
kaiju_addtaxnames = f'{kaiju_path}/kaiju-addTaxonNames -t {kaiju_nodes} -n {kaiju_names} -r superkingdom,phylum,order,class,family,genus,species'

fix_scaffolds_cmd = []
analyze_contigs_cmd = []
addTaxonomy_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    # define file names
    scaffolds_badheaders = f'{assem_dir}/{s}.idba_ud/{s}_scaffold_min1000.fa'
    scaffolds_min1000 = f'{assem_dir}/{s}.idba_ud/{s}_min1000.fa'
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
    gene_calls = f'{assem_dir}/{s}.idba_ud/{s}_gene_calls.fa'
    kaiju_out = f'{assem_dir}/{s}.idba_ud/{s}_kaiju.out'
    kaiju_processed = f'{assem_dir}/{s}.idba_ud/{s}_genes_kaiju.txt'
    
    # fix headers
    cmd = f'{awk} {scaffolds_badheaders} > {scaffolds_min1000}'
    fix_scaffolds_cmd.append(cmd)
    
    # write commands for contigs analysis on cluster
    make_cdb = f'anvi-gen-contigs-database -f {scaffolds_min1000} -o {contigsDB} -n {s} -T 48'
    get_genes = f'anvi-get-sequences-for-gene-calls -c {contigsDB} -o {gene_calls}'
    anvhmms = f'anvi-run-hmms -c {contigsDB} -T 48'
    anvscg = f'anvi-run-scg-taxonomy -c {contigsDB} -T 48' # this didn't actually run because it wasn't set up yet!
    #anvcogs = f'/shared/software/bin/anvi-run-ncbi-cogs -c {contigsDB} -T 48'
    run_kaiju = f'{kaiju} -i {gene_calls} -v -z 48 > {kaiju_out}' 
    cmd = f'sbatch --wrap "{anvio}; {make_cdb}; {get_genes}; {anvhmms}; {anvscg}; {run_kaiju}"'
    analyze_contigs_cmd.append(cmd)
    
    ## write commands for local jobs after cluster jobs
    process_kaiju = f'{kaiju_addtaxnames} -i {kaiju_out} -o {kaiju_processed}' 
    import_kaiju = f'anvi-import-taxonomy-for-genes -i {kaiju_processed} -c {contigsDB} -p kaiju --just-do-it'
    cmd = f'{process_kaiju}; {import_kaiju}'
    addTaxonomy_cmd.append(cmd)

# add to dataframe
df['fix_scaffolds_cmd'] = fix_scaffolds_cmd
df['analyze_contigs_cmd'] = analyze_contigs_cmd
df['addTaxonomy_cmd'] = addTaxonomy_cmd

# save files
df['fix_scaffolds_cmd'].to_csv(f'{outdir}/anvio_0-fix_scaffold_names.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['analyze_contigs_cmd'].to_csv(f'{outdir}/anvio_1-analyze_contigs.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['addTaxonomy_cmd'].to_csv(f'{outdir}/anvio_2-addTaxonomy.sh', index=False, quoting=csv.QUOTE_NONE, header=False, sep='\t')

you must run `conda activate anvio-7.1` in terminal before running the addTaxonomy commands

# SCG taxonomy

In [7]:
df.head()

,sample_id,estscg_taxonomy_cmd
0,ARSTAG_AR_4_27,anvi-estimate-scg-taxonomy -c /groups/banfield...
1,ARSTAG_ARBF_12345_pre,anvi-estimate-scg-taxonomy -c /groups/banfield...
2,ARSTAG_ARBF_1_post,anvi-estimate-scg-taxonomy -c /groups/banfield...
3,ARSTAG_ARBF_2_post,anvi-estimate-scg-taxonomy -c /groups/banfield...
4,ARSTAG_ARBF_3_post,anvi-estimate-scg-taxonomy -c /groups/banfield...


In [65]:
# run-scg failed above because set-up-scg hadn't been run so rerunning

scg_taxonomy_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'

    anvscg = f'anvi-run-scg-taxonomy -c {contigsDB} -T 16' # this didn't actually run because it wasn't set up yet!
    scg_taxonomy_cmd.append(anvscg)

df['scg_taxonomy_cmd'] = scg_taxonomy_cmd

# save files
df['scg_taxonomy_cmd'].to_csv(f'{outdir}/anvio_scg_taxonomy_cmd.sh', index=False, quoting=csv.QUOTE_NONE, header=False, sep='\t')

In [8]:
# estimate SCG taxonomy
# must migrate contigs.dbs to 7.1 and run this with 7.1
estscg_taxonomy_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
    profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/merged/PROFILE.db'
    out = f'{assem_dir}/{s}.idba_ud/anvio_data/{s}_scg_taxonomy.txt'
    anvestscg = f'anvi-estimate-scg-taxonomy -c {contigsDB} -T 16 --metagenome-mode -o {out} -p {profileDB} --compute-scg-coverages -p {profileDB} --compute-scg-coverages -S Ribosomal_S2'
    estscg_taxonomy_cmd.append(anvestscg)

df['estscg_taxonomy_cmd'] = estscg_taxonomy_cmd

# save files
df['estscg_taxonomy_cmd'].to_csv(f'{outdir}/anvio_estscg_taxonomy_cov_cmd.sh', index=False, quoting=csv.QUOTE_NONE, header=False, sep='\t')

# Cross-mapping reads and anvio profile

Build bt2 directories for our min1000 scaffolds to match the Anvio contigs db scaffolds.

In [75]:
with open(f'{outdir}/anvio_3-bt2build.sh', 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_min1000.fa'
        bt2ind = f'{assem_dir}/{s}.idba_ud/bt2/{s}_min1000.fa'
        # index in prep for bowtie2 mapping to get coverage on min1000 scaffolds
        f.write(f'bowtie2-build {scaffolds} {bt2ind}\n')

In [141]:
shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'
anvio = 'conda run -n anvio-7.1'

xmapping_df = []
for srow in df.itertuples():  
    s = srow.sample_id

    scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_min1000.fa'
    bt2base = f'{assem_dir}/{s}.idba_ud/bt2/{s}_min1000.fa'
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
    
    # map against other samples and controls
    for rrow in df.itertuples():
        r = rrow.sample_id
        r_fixed = re.sub('[\.-]', '_', r) # if your sample name has dashes or periods in it, Anvio won't like it.
        
        r1 = f'{reads_dir}/{r}_trim_clean.PE.1.fastq.gz'
        r2 = f'{reads_dir}/{r}_trim_clean.PE.2.fastq.gz'        
        raw_bam = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}-raw.bam'
        filtered_bam_raw = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}-raw-filt.bam'
        bam = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}.bam'
        
        if 'CTRL' in r or 'CONTROL' in r:
            
            # filter mapping - for decontamination, we want to use filtered mappings
            map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {raw_bam}; ' \
                      f'reformat.sh in={raw_bam} out={filtered_bam_raw} editfilter=2 threads=48; rm {raw_bam}"'
        
        else:
            map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {filtered_bam_raw}"'
        
        # map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {filtered_bam_raw}"'
        
        # process mapping
        initbam = f'anvi-init-bam {filtered_bam_raw} -o {bam}; rm {filtered_bam_raw}'
        
        # anvi-profile
        profile_out = f'{assem_dir}/{s}.idba_ud/anvio_data/{r}_profile' # check this
        anvip_cmd = f'sbatch --wrap "{anvio} anvi-profile --min-contig-length 1000 --skip-SNV-profiling -T 48 -i {bam} -c {contigsDB} -o {profile_out} -S {r_fixed}"'
        
        ## append all commands to table
        xmapping_df.append([s, r, map_cmd, initbam, anvip_cmd])
xmapping_df = pd.DataFrame.from_records(xmapping_df, columns=['assembly', 'reads', 'map', 'initbam', 'anvi-profile'])

In [142]:
xmapping_df.to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/crossmapping.csv', index=False)

In [113]:
# example of filtering the xmapping_df to get just the rows you want
pos = xmapping_df[(xmapping_df.assembly == 'AWTP2_CTRL_POS-POWER9_EXTRACTION_0') 
                  & ((xmapping_df.reads.str.contains('POS')) | (xmapping_df.reads.str.contains('BLANK')))]

pos['map'].to_csv(f'{outdir}/anvio_xmap_pos-controls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
pos['initbam'].to_csv(f'{outdir}/anvio_initbam_pos-controls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
pos['anvi-profile'].to_csv(f'{outdir}/anvio_anvip_pos-controls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

Based on MASH distances between samples, decide which samples to cross-map against one another. A rule of thumb is that cross-mapping beyond 12 samples likely will not improve your bins.

In [18]:
df1 = pd.read_csv(f'{outdir}/rerun_profile.txt', sep=',', names=['assembly','reads','map','initbam','anvi-profile'])
df1['anvi-profile'] = df1['anvi-profile'].str.replace('conda run -n anvio-7.1 ', '')
df1['anvi-profile'].to_csv(f'{outdir}/rerun_profile.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

<ipython-input-18-685a643f1953>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['anvi-profile'] = df1['anvi-profile'].str.replace('conda run -n anvio-7.1 ', '')


# Anvi-merge to combine crossmappings into a single profile

In [ ]:
with open(f'{outdir}/anvio_merge.sh', 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
        profiles = f'{assem_dir}/{s}.idba_ud/anvio_data/*_profile/PROFILE.db'
        out_profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/merged'
        merge_cmd = f'anvi-merge -c {contigsDB} -p {profiles} -o {out_profileDB} -S {s}' #  --skip-hierarchical-clustering # --enforce-hierarchical-clustering
        f.write(f'{merge_cmd}\n')

# Binning

Note: you may want to tell concoct how many bins to create based on the counts of the SCGs in your samples.

To do this, you could add a column to your df with that information and then uncomment the lines below (and include in the concoct command)

One of the Anvi'o tutorials suggests it's better to underestimate the number of genomes since it is easier to split contaminated bins with anvi-refine than to combine fragmented bins.

In [5]:
outfile = f'{outdir}/anvio_concoct.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        # c = row.scg_count
        profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/merged/PROFILE.db'
        contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
        out = f'{assem_dir}/{s}.idba_ud/anvio_data'

        get_cococt_input = f'anvi-export-splits-and-coverages -p {profileDB} -c {contigsDB} -o {out} -O {s} --splits-mode'
        run_concoct = f'concoct --coverage_file {out}/{s}-COVs.txt --composition_file {out}/{s}-SPLITS.fa -b {out}/{s}_concoct -r 150 -t 10' #-c {c}
        csv_to_tsv = f'sed "s/,/\\tbin_/g" {out}/{s}_concoct_clustering_gt1000.csv > {out}/{s}_concoct_clustering_gt1000.tsv'
        import_collection = f'anvi-import-collection {s}_concoct_clustering_gt1000.tsv -p {profileDB} -c {contigsDB} -C concoct_noest'

        f.write(f'{get_cococt_input}; {run_concoct}; {csv_to_tsv}; {import_collection}\n')

Here are the parameters Anvio is using for metabat (based on the log file it created)

metabat2 -i /tmp/tmpcs7pncgj/sequence_contigs.fa -a /tmp/tmpcs7pncgj/contig_coverages.txt -o /tmp/tmpcs7pncgj/METABAT_ --cvExt -l

MetaBAT 2 (v2.12.1) using minContig 2500, minCV 1.0, minCVSum 1.0, maxP 95%, minS 60, and maxEdges 200

Note: it looks like metabat can also take into account the coverage variances for each contig in each sample, which might improve binning.  
Anvi'o doesn't send that input to metabat, so we would have to separately profile all the bam files with jgi_summarize_bam_contig_depths and feed it to metabat ourselves.  If the bins aren't good enough without this, then we can try it!

In [27]:
anvio = 'conda run -n anvio-7.1'
outfile = f'{outdir}/anvio_metabat.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        # c = row.scg_count
        profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/merged/PROFILE.db'
        contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
        f.write(f'sbatch --wrap "{anvio} anvi-cluster-contigs -c {contigsDB} -p {profileDB} --driver metabat2 -C metabat -T 48 --just-do-it"\n')

In [7]:
anvio = '~/github/anvio/bin'
outfile = f'{outdir}/anvio_metabat_split.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        # c = row.scg_count
        profileDB = f'{assem_dir}/{s}.idba_ud/split/none/PROFILE.db'
        contigsDB = f'{assem_dir}/{s}.idba_ud/split/none/CONTIGS.db'
        f.write(f'sbatch --wrap "{anvio}/anvi-cluster-contigs -c {contigsDB} -p {profileDB} --driver metabat2 -C metabat -T 48 --just-do-it"\n')

## running snakemake binning pipeline

In [10]:
# extract contigs
outfile = f'{outdir}/anvio_export_decontam_contigs.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        if 'CTRL' in s:
            continue
        contigsDB = f'{assem_dir}/{s}.idba_ud/split/none/CONTIGS.db'
        fasta = f'{assem_dir}/{s}.idba_ud/split/none/{s}_contigs.fa'
        f.write(f'anvi-export-contigs -c {contigsDB} -o {fasta}\n')

In [10]:
# make samples.tsv config file for snakemake run

df_snakemake = df[['sample_id']].copy().sort_values('sample_id')
df_snakemake = df_snakemake.rename(columns={'sample_id': 'sample_name'})


df_snakemake['assembly_path'] = df_snakemake.apply(lambda x: f'{assem_dir}/{x.sample_name}.idba_ud/split/none/{x.sample_name}_contigs.fa', axis=1)
df_snakemake['forward_read_path'] = df_snakemake.apply(lambda x: f'{reads_dir}/{x.sample_name}_trim_clean.PE.1.fastq.gz', axis=1) 
df_snakemake['reverse_read_path'] = df_snakemake.apply(lambda x: f'{reads_dir}/{x.sample_name}_trim_clean.PE.2.fastq.gz', axis=1)

# just MF_RO
df_mfro = df_snakemake[df_snakemake.sample_name.str.contains('MF') | 
                        df_snakemake.sample_name.str.contains('RO2_BIOFILM')].copy()
df_mfro = df_mfro[~df_mfro.sample_name.str.contains('CTRL')]
df_mfro.to_csv(f'{outdir}/binning_snakemake_mfro_samples.tsv', sep='\t', index=False)

# # just RO
# df_ro = df_snakemake[df_snakemake.sample_name.str.contains('RO2_BULK')].copy()
# df_ro = df_ro[~df_ro.sample_name.str.contains('CTRL')]
# df_ro.to_csv(f'{outdir}/binning_snakemake_ro_samples.tsv', sep='\t', index=False)

# snakemake -j10 --use-conda --cluster "sbatch -J ggbin" --cluster-cancel scancel --latency-wait 60 --rerun-incomplete --dry-run

In [66]:
df_snakemake.assembly_path.values[0]

'/groups/banfield/projects/industrial/nelson_lab/awtp2/assemblies/AWTP2_CTRL_BLANK_EXTRACTION_0.idba_ud/split/none/CONTIGS.db'

## Check bin quality

In [5]:
bins_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/workflow/ggBin/results'
samples = ['AWTP2_SAMPLE_RO2_BULK_3', 'AWTP2_SAMPLE_RO2_BULK_4', 'AWTP2_SAMPLE_RO2_BULK_5', 'AWTP2_SAMPLE_RO2_BULK_6']

outfile = f'{outdir}/checkm2_dastool_ggBin.sh'
with open(outfile, 'w') as f:
    for s in samples:
        bins = f'{bins_dir}/{s}/bins/dastool/results/{s}.asm.fa.dastool/bins/'
        output = f'{bins_dir}/{s}/bins/dastool/results/{s}.asm.fa.dastool/checkm2'
        f.write(f'sbatch --wrap "checkm2 predict --input {bins} --output-directory {output} --threads 48 -x .fa"\n')

# Annotation with KEGG KOFams

In [9]:
anvio = 'conda run -n anvio-7.1'
outfile = f'{outdir}/anvio_run_kegg.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
        f.write(f'sbatch --wrap "{anvio} anvi-run-kegg-kofams -c {contigsDB} -T 48"\n')

# For AWTP2 decontaminated

In [10]:
# for some reason the RNA HMMs were not run on these dbs
anvio = 'conda run -n anvio-7.1'
anvihmms_rerun = []

for row in df.itertuples():
    s = row.sample_id
    contigsDB = f'{assem_dir}/{s}.idba_ud/split/none/CONTIGS.db'
    anvhmms = f'anvi-run-hmms -c {contigsDB} -T 48 -I Ribosomal_RNA_18S,Ribosomal_RNA_28S,Ribosomal_RNA_16S,Ribosomal_RNA_5S,Ribosomal_RNA_23S --just-do-it'
    cmd = f'sbatch --wrap "{anvio} {anvhmms}"'
    
    anvihmms_rerun.append(cmd)
    
# add to dataframe

df['anvihmms_rerun'] = anvihmms_rerun

# save files
df['anvihmms_rerun'].to_csv(f'{outdir}/anvio_hmms_rerun.sh', index=False, quoting=csv.QUOTE_NONE, sep='\t', header=False)

In [45]:
# add SCG count per metagenome to table to provide CONCOCT with an estimate number of bins to create
scg_counts = []
files = glob('/Users/rosekantor/data/metagenomics_mentoring/awtp2/phylogenetic_analyses/scg_taxonomy/*.txt')
for f in files:
    sample_id = re.match(r'.+\/(.+)_scg_taxonomy.txt', f).group(1)
    df_scg = pd.read_csv(f, sep='\t')
    scg_count = (len(df_scg))
    scg_counts.append([sample_id, scg_count])

scg_counts = pd.DataFrame.from_records(scg_counts, columns=('sample_id', 'scg_count'))
df = df.merge(scg_counts, how='left', on='sample_id')

In [57]:
# run with anvio-dev version
# split up commands so that concoct can be run on cluster to avoid memory issues and using all the threads

concoct_input = []
run_concoct = []
import_concoct = []

for row in df.itertuples():
    s = row.sample_id
    c = row.scg_count
    profileDB = f'{assem_dir}/{s}.idba_ud/split/none/PROFILE.db'
    contigsDB = f'{assem_dir}/{s}.idba_ud/split/none/CONTIGS.db'
    out = f'{assem_dir}/{s}.idba_ud/split/none'

    cococt_input_cmd = f'anvi-export-splits-and-coverages -p {profileDB} -c {contigsDB} -o {out} -O {s} --splits-mode'
    concoct_input.append(cococt_input_cmd)
    
    run_concoct_cmd = f'sbatch --wrap "concoct --coverage_file {out}/{s}-COVs.txt --composition_file {out}/{s}-SPLITS.fa -b {out}/{s}_concoct -r 150 -t 48 -c {c}"' 
    run_concoct.append(run_concoct_cmd)
    
    csv_to_tsv = f'sed "s/,/\\tbin_/g" {out}/{s}_concoct_clustering_gt1000.csv > {out}/{s}_concoct_clustering_gt1000.tsv'
    import_collection = f'anvi-import-collection {out}/{s}_concoct_clustering_gt1000.tsv -p {profileDB} -c {contigsDB} -C concoct'
    
    import_concoct.append(f'{csv_to_tsv}; {import_collection}')
    
# add to dataframe

df['concoct_input'] = concoct_input
df['run_concoct'] = run_concoct
df['import_concoct'] = import_concoct

# save files
df[~df.sample_id.str.contains('CTRL')]['concoct_input'].to_csv(f'{outdir}/anvio_concoct_input.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df[~df.sample_id.str.contains('CTRL')]['run_concoct'].to_csv(f'{outdir}/anvio_run_concoct.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df[~df.sample_id.str.contains('CTRL')]['import_concoct'].to_csv(f'{outdir}/anvio_import_concoct.sh', index=False, quoting=csv.QUOTE_NONE, sep='\t', header=False)

In [58]:
anvio = 'conda run -n anvio-7.1'
outfile = f'{outdir}/anvio_run_kegg_split.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        contigsDB = f'{assem_dir}/{s}.idba_ud/split/none/CONTIGS.db'
        f.write(f'sbatch --wrap "{anvio} anvi-run-kegg-kofams -c {contigsDB} -T 48"\n')

# mapping and Anvio profiling for dereplicated refined MAGs

In [39]:
# mapping to dereplicated refined RO MAGs

shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'
anvio = 'conda run -n anvio-dev'

mapping_df = []

drep_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_RO2_BULK'
scaffolds = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_RO2_BULK/RO_drep_refined.fasta'
bt2base = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_RO2_BULK/bt2/RO_drep_refined.fasta'
contigsDB = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_RO2_BULK/RO_mags.db'
s = 'dRep_RO2_BULK'

for row in df.itertuples():
    r = row.sample_id
    r_fixed = re.sub('[\.-]', '_', r) # if your sample name has dashes or periods in it, Anvio won't like it.
    r1 = f'{reads_dir}/{r}_trim_clean.PE.1.fastq.gz'
    r2 = f'{reads_dir}/{r}_trim_clean.PE.2.fastq.gz'        
    raw_bam = f'{drep_dir}/{s}-vs-{r}-raw.bam'
    filtered_bam_raw = f'{drep_dir}/{s}-vs-{r}-raw-filt.bam'
    bam = f'{drep_dir}/{s}-vs-{r}.bam'

    if 'CTRL' in r or 'CONTROL' in r:
        continue
    
    # filter mapping to 5 mismatches (removed shrinksam so anvio can tell how many reads mapped)
    map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | sambam > {raw_bam}; ' \
              f'reformat.sh in={raw_bam} out={filtered_bam_raw} editfilter=5 threads=48; rm {raw_bam}"'

    # process mapping
    initbam = f'anvi-init-bam {filtered_bam_raw} -o {bam}; rm {filtered_bam_raw}'

    # anvi-profile
    profile_out = f'{drep_dir}/anvio_data/{r}_profile' # check this
    anvip_cmd = f'sbatch --wrap "{anvio} anvi-profile --min-contig-length 1000 --skip-SNV-profiling -T 48 -i {bam} -c {contigsDB} -o {profile_out} -S {r_fixed}"'

    ## append all commands to table
    mapping_df.append([r, map_cmd, initbam, anvip_cmd])
mapping_df = pd.DataFrame.from_records(mapping_df, columns=['reads', 'map', 'initbam', 'anvi-profile'])

In [11]:
# mapping to dereplicated refined RO and MF MAGs

shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'
anvio = 'conda run -n anvio-dev'

mapping_df = []

drep_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes'
scaffolds = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/contigs.fasta'
bt2base = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/bt2/contigs.fasta'
contigsDB = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/contigs.db'
s = 'dRep_membranes'

for row in df.itertuples():
    r = row.sample_id
    r_fixed = re.sub('[\.-]', '_', r) # if your sample name has dashes or periods in it, Anvio won't like it.
    r1 = f'{reads_dir}/{r}_trim_clean.PE.1.fastq.gz'
    r2 = f'{reads_dir}/{r}_trim_clean.PE.2.fastq.gz'        
    raw_bam = f'{drep_dir}/{s}-vs-{r}-raw.bam'
    filtered_bam_raw = f'{drep_dir}/{s}-vs-{r}-raw-filt.bam'
    bam = f'{drep_dir}/{s}-vs-{r}.bam'

    if 'CTRL' in r or 'CONTROL' in r:
        continue
    
    # filter mapping to 5 mismatches (removed shrinksam so anvio can tell how many reads mapped)
    map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | sambam > {raw_bam}; ' \
              f'reformat.sh in={raw_bam} out={filtered_bam_raw} editfilter=5 threads=48; rm {raw_bam}"'

    # process mapping
    initbam = f'anvi-init-bam {filtered_bam_raw} -o {bam}; rm {filtered_bam_raw}'

    # anvi-profile
    profile_out = f'{drep_dir}/anvio_data/{r}_profile' # check this
    # note: this time, doing SNV profiling, so removed --skip-SNV-profiling
    anvip_cmd = f'sbatch --wrap "{anvio} anvi-profile --min-contig-length 1000 -T 48 -i {bam} -c {contigsDB} -o {profile_out} -S {r_fixed}"'

    ## append all commands to table
    mapping_df.append([r, map_cmd, initbam, anvip_cmd])
mapping_df = pd.DataFrame.from_records(mapping_df, columns=['reads', 'map', 'initbam', 'anvi-profile'])

In [12]:
mapping_df['map'].to_csv(f'{outdir}/map_to_drep_membranes.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
mapping_df['initbam'].to_csv(f'{outdir}/initbam_drep_membranes.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
mapping_df['anvi-profile'].to_csv(f'{outdir}/profile_drep_membranes.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

# Running InStrain on the mappings to dereplicated membrane MAGs

In [15]:
# test on one genome (only ran on 2 samples for this test but generated commands for all)
drep_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes'
scaffolds = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/contigs.fasta'
scaffold2bin = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/scaffold_to_bin.tsv'
contigs = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/contigs.fasta'

# for test run, using --scaffolds_to_profile to include just the aquabacterium
# note that the bam files generated above were already filtered to allow only 5 mismatches per read, consider rerunning

with open(f'{outdir}/instrain_aquabacterium3test.sh', "w") as f:
    for row in df.itertuples():        
        r = row.sample_id
        r_fixed = re.sub('[\.-]', '_', r)
        outprefix = f'{drep_dir}/instrain/{r_fixed}'
        
        # only profile bins with mappings of the MF and RO samples
        if 'RO2' in r or 'MFCOMB' in r:        
            bam = f"{drep_dir}/dRep_membranes-vs-{r_fixed}.bam"
            cmd = f'sbatch --wrap "inStrain profile -o {outprefix} -p 48 -s {scaffold2bin} --scaffolds_to_profile {drep_dir}/aquabacterium_3_scaffolds.txt {bam} {contigs}"'
            f.write(cmd+"\n")
f.close()

In [20]:
# test on one genome (only ran on 2 samples for this test but generated commands for all)
drep_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes'
scaffolds = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/contigs.fasta'
scaffold2bin = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/scaffold_to_bin.tsv'
contigs = '/groups/banfield/projects/industrial/nelson_lab/awtp2/dRep_membranes/contigs.fasta'

# for test run, using --scaffolds_to_profile to include just the aquabacterium
# note that the bam files generated above were already filtered to allow only 5 mismatches per read, consider rerunning

with open(f'{outdir}/instrain_drep_membrane_genomes.sh', "w") as f:
    for row in df.itertuples():        
        r = row.sample_id
        r_fixed = re.sub('[\.-]', '_', r)
        outprefix = f'{drep_dir}/instrain/{r_fixed}'
        
        # drop controls
        if 'CTRL' in r or 'CONTROL' in r:
            continue
        
        # only profile bins with mappings of the MF and RO samples
        if 'RO2' in r or 'MFCOMB' in r:        
            bam = f"{drep_dir}/dRep_membranes-vs-{r_fixed}.bam"
            cmd = f'sbatch --wrap "inStrain profile -o {outprefix} -p 48 -s {scaffold2bin} \
--min_scaffold_reads 10  --min_genome_coverage 1 --min_read_ani 0.96 \
{bam} {contigs}"'
            f.write(cmd+"\n")
f.close()

# Pull 16S, 18S, SCG for taxonomy

- who is present and how abundant? across how many samples?
- do we have bins for them?
- does this match to the 16S V4 amplicon data

1. pull 16S and 18S
2. cluster at 100% across samples
3. classify with SILVA's on line SINA classifier
4. compile into table

In [6]:
# must migrate contigs.dbs to 7.1 and run this with 7.1
anvi_pull_18S_cmd = []
anvi_pull_16S_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    contigsDB = f'{assem_dir}/{s}.idba_ud/split/none/CONTIGS.db'
    profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/{s}_profile/PROFILE.db'
    
    out = f'{assem_dir}/{s}.idba_ud/split/none/18S.fasta'
    anvi_pull_18S = f'anvi-get-sequences-for-hmm-hits -c {contigsDB} --hmm-sources Ribosomal_RNA_18S -o {out}'
    anvi_pull_18S_cmd.append(anvi_pull_18S)
    
    out = f'{assem_dir}/{s}.idba_ud/split/none/16S.fasta'
    anvi_pull_16S = f'anvi-get-sequences-for-hmm-hits -c {contigsDB} --hmm-sources Ribosomal_RNA_16S -o {out}'
    anvi_pull_16S_cmd.append(anvi_pull_16S)

df['anvi_pull_18S_cmd'] = anvi_pull_18S_cmd
df['anvi_pull_18S_cmd'].to_csv(f'{outdir}/anvio_pull_18S_cmd.sh', index=False, quoting=csv.QUOTE_NONE, header=False, sep='\t')

df['anvi_pull_16S_cmd'] = anvi_pull_16S_cmd
df['anvi_pull_16S_cmd'].to_csv(f'{outdir}/anvio_pull_16S_cmd.sh', index=False, quoting=csv.QUOTE_NONE, header=False, sep='\t')

In [12]:
# estimate SCG taxonomy on decontaminated data
anvio = 'conda run -n anvio-dev'

outfile = f'{outdir}/anvio_runscg_split.sh'
with open(outfile, 'w') as f:

    for row in df[~df.sample_id.str.contains('CTRL')].itertuples():
        s = row.sample_id
        contigsDB = f'{assem_dir}/{s}.idba_ud/split/none/CONTIGS.db'
        profileDB = f'{assem_dir}/{s}.idba_ud/split/none/PROFILE.db'
        anvrunscg = f'sbatch --wrap "{anvio} anvi-run-scg-taxonomy -c {contigsDB} -T 48"'
        f.write(anvrunscg+'\n')

outfile = f'{outdir}/anvio_estscg_split.sh'
with open(outfile, 'w') as f:

    for row in df[~df.sample_id.str.contains('CTRL')].itertuples():
        s = row.sample_id
        contigsDB = f'{assem_dir}/{s}.idba_ud/split/none/CONTIGS.db'
        profileDB = f'{assem_dir}/{s}.idba_ud/split/none/PROFILE.db'
        out = f'{assem_dir}/{s}.idba_ud/split/none/{s}_scg_taxonomy.txt'
        anvestscg = f'sbatch --wrap "{anvio} anvi-estimate-scg-taxonomy -c {contigsDB} -T 48 --metagenome-mode -o {out} -p {profileDB} --compute-scg-coverages -p {profileDB} --compute-scg-coverages -S Ribosomal_S3_C"'
        f.write(anvestscg+'\n')

# Read mapping to human virus genomes

clustered polyomaviruses and adenoviruses at 99% ID before mapping so that we can threshold coverage depth and hopefully avoid having coverage split across multiple genomes...

usearch -sortbylength polyomavirus_adenovirus.fasta -fastaout polyomavirus_adenovirus.sorted.fasta

usearch -cluster_fast polyomavirus_adenovirus.sorted.fasta -id 0.99 -uc polyomavirus_adenovirus.uc -centroids polyomavirus_adenovirus.centroids.fasta -threads 2

In [22]:
bt2 = '/shared/software/bin/bowtie2'
virus_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/virus_analysis'

virus_mapping_df = []

outfile = f'{outdir}/virus_map.sh'
with open(outfile, 'w') as f:
    
    for row in df.itertuples():  
        s = row.sample_id

        if 'CTRL' in s or 'CONTROL' in s:
            continue

        bt2base = f'{virus_dir}/bt2/polyomavirus_adenovirus.centroids.fasta'
        r1 = f'{reads_dir}/{s}_trim_clean.PE.1.fastq.gz'
        r2 = f'{reads_dir}/{s}_trim_clean.PE.2.fastq.gz'        
        raw_bam = f'{virus_dir}/{s}-vs-polyomavirus_adenovirus_centroids-raw.bam'

        map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} --no-unal | sambam > {raw_bam}"'
        
        f.write(map_cmd+'\n')

In [32]:
# sort and index virus mappings
virus_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/virus_analysis'

outfile = f'{outdir}/virus_count.sh'
with open(outfile, 'w') as f:
    
    for row in df.itertuples():  
        s = row.sample_id

        if 'CTRL' in s or 'CONTROL' in s:
            continue
        raw_bam = f'{virus_dir}/{s}-vs-polyomavirus_adenovirus_centroids-raw.bam'
        filtered_bam = f'{virus_dir}/{s}-vs-polyomavirus_adenovirus_centroids.bam'
        sorted_bam = f'{virus_dir}/{s}-vs-polyomavirus_adenovirus_centroids.sorted.bam'

        # calculate coverage breadth at >=1x depth for each target, ignore reads shorter than 75 bp
        # then say it's present, in which case, count
        samtools_cmds = f'samtools sort {raw_bam} > {sorted_bam}; '\
                        f'samtools index {sorted_bam}; '\
                        f'samtools idxstats {sorted_bam} > {virus_dir}/{s}-vs-polyomavirus_adenovirus_centroids.stats.txt; '\
                        f'samtools coverage --min-read-len 75 {sorted_bam} > {virus_dir}/{s}-vs-polyomavirus_adenovirus_centroids.cov.txt'

        f.write(samtools_cmds+'\n')

# ARG analysis

Loaded CARD db including wildcard, according to this guide: https://github.com/arpcard/rgi/blob/master/docs/rgi_bwt.rst

Steps
1. filter to remove reads shorter than 75 bp (bbduk)
2. Count how many reads remain in each sample (seqkit)
3. subsample filtered reads to even depth (seqtk)
4. run rgi bwt analysis to map to CARD and wildCARD using kma mapping and parse output

In [35]:
# filter reads
reads_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/reads/trimmed'
filtered_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/reads/filtered'

outfile = f'{outdir}/filter_reads75_forARGmapping.sh'
with open(outfile, 'w') as f:

    for row in df.itertuples():
        s = row.sample_id
        
        if 'CTRL' in s or 'CONTROL' in s:
            continue
            
        r1 = f'{reads_dir}/{s}_trim_clean.PE.1.fastq.gz'
        r2 = f'{reads_dir}/{s}_trim_clean.PE.2.fastq.gz'
        out1 = f'{filtered_dir}/{s}_filtered.PE.1.fastq.gz'
        out2 = f'{filtered_dir}/{s}_filtered.PE.2.fastq.gz'

        cmd = f'sbatch --wrap "bbduk.sh in1={r1} in2={r2} out1={out1} out2={out2} qtrim=r trimq=10 minlen=75 threads=48"'
        
        f.write(cmd+'\n')

from within the filtered reads dir, run:

`seqkit stat *.fastq.gz > seqkit_output.tsv`

In [36]:
# filter reads
filtered_dir = '/groups/banfield/projects/industrial/nelson_lab/awtp2/reads/filtered'
subsampled = '/groups/banfield/projects/industrial/nelson_lab/awtp2/reads/subsampled'

outfile = f'{outdir}/subsample_reads_forARGmapping.sh'
with open(outfile, 'w') as f:

    for row in df.itertuples():
        s = row.sample_id
        
        if 'CTRL' in s or 'CONTROL' in s:
            continue

        r1 = f'{filtered_dir}/{s}_filtered.PE.1.fastq.gz'
        r2 = f'{filtered_dir}/{s}_filtered.PE.2.fastq.gz'
        sub1 = f'{subsampled}/{s}_sub.PE.1.fastq.gz'
        sub2 = f'{subsampled}/{s}_sub.PE.2.fastq.gz'

        cmd = f'seqtk sample -s100 {r1} 9000000 > {sub1}; seqtk sample -s100 {r2} 9000000 > {sub2}'
        
        f.write(cmd+'\n')

# Next steps

Annotation:
- KEGG HMMs
- Pfam HMMs (if desired)
- estimate metabolism (curious about this)

Additional questions:
- annotate with special HMMs (e.g. ARGs, METABOLIC)
- investigate metabolisms of key organisms of interest
- Virsorter2 to look for phage
- look for pathogens
- phylogenetic trees for key organisms of interest
- assess growth via iRep